# AI Manga Upscale Colab

<br>Upscale black-and-white and color manga pages by 4x using AI.
<br><br>You may want to use the post-production toolset (downscaler, page renamer, cbz creator) - [AI Manga Upscale Colab Post-Production Toolset](https://colab.research.google.com/github/Spladenly/ai-manga-upscale-colab/blob/main/post-prod.ipynb).<br><br>

> When running the script, you will need to grant access to your Google Drive. This permission will be used exclusively to access the folder: "/ESRGAN". No other personal folders or files will be used.

<br>

Folder structure on Google Drive:

/ESRGAN/bw (black-and-white manga pages go here) *
<br>/ESRGAN/color (color manga pages go here) *
<br>/ESRGAN/output (upscaled manga pages will appear here)
<br><br>Manga pages can also be uploaded to subfolders.
<br><br> * Supported files: bmp, dib, jpeg, jpg, jpe, jp2, png, webp, pbm, pgm, ppm, pxm, pnm, pfm, sr, ras, tiff, tif, exr, hdr, pic, gif, tga.<br><br>

> Google Drive has a delay. If you've just uploaded or deleted files -- Google Colab may not see the changes for up to 1 minute. It works the same way in the other direction: if processing has just finished -- you won't see new files on Google Drive right away. Please wait.

<br>To begin processing -- upload manga pages to previously mentioned folder(s), and click the run button bellow

In [ ]:
# @title Manga Upscaler { display-mode: "form" }

import os
import torch
from google.colab import drive
from pathlib import Path

def check_connect_gdrive():
  if not os.path.exists("/content/gdrive/MyDrive"):
    print("Google Drive connection in progress...")
    drive.mount("/content/gdrive")

def check_clone_esrgan():
  if not os.path.exists("ESRGAN"):
    !git clone https://github.com/Spladenly/ESRGAN

def init_dirs():
  Path("/content/gdrive/MyDrive/ESRGAN").mkdir \
   (parents=True, exist_ok=True)
  Path("/content/gdrive/MyDrive/ESRGAN/bw").mkdir \
   (parents=True, exist_ok=True)
  Path("/content/gdrive/MyDrive/ESRGAN/color").mkdir \
   (parents=True, exist_ok=True)
  Path("/content/gdrive/MyDrive/ESRGAN/output").mkdir \
   (parents=True, exist_ok=True)

def dir_contains_files(path):
  for root, dirs, files in os.walk(path):
    if files:
      return True
  return False

def ai_process_bw():
  !python ESRGAN/upscale.py -se -i /content/gdrive/MyDrive/ESRGAN/bw \
  -o /content/gdrive/MyDrive/ESRGAN/output \
  ESRGAN/models/4x_eula_digimanga_bw_v2_nc1_307k.pth

def ai_process_color():
  !python ESRGAN/upscale.py -se -i /content/gdrive/MyDrive/ESRGAN/color \
  -o /content/gdrive/MyDrive/ESRGAN/output \
  ESRGAN/models/4x-AnimeSharp.pth

def main():
  print("[AI Manga Upscale Colab] Manga Upscaler")

  if not torch.cuda.is_available():
    print("This session doesn't have a GPU.\n To connect a GPU, click:\n'Edit' -> 'Notebook settings' -> 'Hardware accelerator' = GPU; 'GPU type' = T4.\nAfter that, run this script again.")
    return

  check_connect_gdrive()
  check_clone_esrgan()
  init_dirs()

  status = 0

  if dir_contains_files("/content/gdrive/MyDrive/ESRGAN/bw"):
    status += 1
    print("Starting black-and-white upscale subprocess...")
    ai_process_bw()

  if dir_contains_files("/content/gdrive/MyDrive/ESRGAN/color"):
    status += 1
    print("Starting color upscale subprocess...")
    ai_process_color()

  if status == 0:
    print("No pages were found in the following directories: '/ESRGAN/bw' and '/ESRGAN/color' on your Google Drive.\nPlease upload manga pages there, and run this script again.")
  else:
    print("The processing has been finished. The result can be downloaded from '/ESRGAN/output' on your Google Drive.\nTo process additional pages, run this script again. If you don't plan to process additional pages in the near future, please close the current session.\nThis can be done by clicking on: the inverted triangle (next to the 'RAM' and 'Disk' labels in the upper right corner) -> 'Disable and remove runtime'.\nThis will unlock the resources reserved for this session for other users.")

if __name__ == "__main__":
  main()

You may also upload CBZs and ZIPs instead of images. Use the following script to extract them before running Upscale Manga. After the extraction, the original files will be moved to recycle bin.

In [ ]:
# @title Extractor { display-mode: "form" }

import os
import shutil
import zipfile
from google.colab import drive

def check_connect_gdrive():
  if not os.path.exists("/content/gdrive/MyDrive"):
    print("Google Drive connection in progress...")
    drive.mount("/content/gdrive")

def unpack(path):
  zip_files = [f for f in os.listdir(path) if f.endswith(".zip") or \
               f.endswith(".cbz")]

  print(f"Found {len(zip_files)} file(s).")

  for zip_file in zip_files:
      print(f"Extracting {zip_file}...")

      zip_path = os.path.join(path, zip_file)

      if not os.path.exists(os.path.splitext(zip_path)[0]):
        extract_folder = os.path.join(path, \
                                        os.path.splitext(zip_file)[0])
        os.makedirs(extract_folder, exist_ok=True)

        with zipfile.ZipFile(zip_path, "r") as zf:
            zf.extractall(extract_folder)

        os.remove(zip_path)
      else:
         print("The folder already exists -- skipping.")

def main():
  print("[AI Manga Upscale Colab] Extractor")

  check_connect_gdrive()

  status = 0
  bw_path = "/content/gdrive/MyDrive/ESRGAN/bw"
  color_path = "/content/gdrive/MyDrive/ESRGAN/color"

  if os.path.exists(bw_path):
    status += 1
    print(f"Scanning: {bw_path}")
    unpack(bw_path)

  if os.path.exists(color_path):
    status += 1
    print(f"Scanning: {color_path}")
    unpack(color_path)

  if status == 0:
    print("Both 'bw' and 'color' directories don't exist.")

  print("Done.")

if __name__ == "__main__":
  main()